In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/josh-complete/processed/transcripts/1021370_seg80.txt
/kaggle/input/josh-complete/processed/transcripts/857737_seg39.txt
/kaggle/input/josh-complete/processed/transcripts/319431_seg43.txt
/kaggle/input/josh-complete/processed/transcripts/255349_seg3.txt
/kaggle/input/josh-complete/processed/transcripts/583533_seg30.txt
/kaggle/input/josh-complete/processed/transcripts/583544_seg23.txt
/kaggle/input/josh-complete/processed/transcripts/856801_seg76.txt
/kaggle/input/josh-complete/processed/transcripts/243702_seg15.txt
/kaggle/input/josh-complete/processed/transcripts/365059_seg19.txt
/kaggle/input/josh-complete/processed/transcripts/856801_seg28.txt
/kaggle/input/josh-complete/processed/transcripts/979497_seg58.txt
/kaggle/input/josh-complete/processed/transcripts/1021370_seg25.txt
/kaggle/input/josh-complete/processed/transcripts/319126_seg34.txt
/kaggle/input/josh-complete/processed/transcripts/979497_seg24.txt
/kaggle/input/josh-complete/processed/transcripts/629904_seg4

In [2]:
# -------------------------------------------------
# CELL 1: Install (PEFT 0.6.0 fixes the bug)
# -------------------------------------------------
print("Installing packages...")
!pip uninstall -y peft -q
!pip install -q peft==0.6.0
!pip install -q transformers==4.36.2 datasets==2.15.0 accelerate==0.25.0 soundfile librosa
print("Installation complete!\n")

Installing packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.8 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:00:00:0100:01
E

In [3]:
# -------------------------------------------------
# CELL 2: Imports + GPU check
# -------------------------------------------------
import os
import json
import numpy as np
import torch
import soundfile as sf
import librosa
from datasets import load_from_disk, DatasetDict
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from dataclasses import dataclass
from typing import Any, Dict, List

import warnings
warnings.filterwarnings('ignore')

print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB\n")

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
2025-11-17 06:56:52.756744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763362612.950447      48

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

GPU: Tesla P100-PCIE-16GB
VRAM: 15.89 GB



In [4]:
# -------------------------------------------------
# CELL 3: Speed-boost flags (P100 loves TF32)
# -------------------------------------------------
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
print("TF32 + cuDNN benchmark enabled\n")

TF32 + cuDNN benchmark enabled



In [28]:
# -------------------------------------------------
# CELL 3: CONFIG – P100 SAFE & FAST
# -------------------------------------------------
class Config:
    HF_DATASET_PATH   = "/kaggle/input/josh-complete/processed/hf_dataset_segments"
    AUDIO_SEGMENTS_PATH = "/kaggle/input/josh-complete/processed/segments"
    OUTPUT_DIR        = "/kaggle/working/whisper_lora_output"

    MODEL_NAME = "openai/whisper-small"
    LANGUAGE   = "hi"

    USE_FULL_DATASET = True
    NUM_EPOCHS = 3
    BATCH_SIZE = 24                    
    GRAD_ACC   = 1
    LR         = 1e-3
    WARMUP     = 0.1

    LORA_R      = 32
    LORA_ALPHA  = 64
    LORA_DROP   = 0.0                  
    TARGET_MOD  = ["q_proj", "v_proj", "k_proj", "o_proj"]

    LOG_STEPS = 25
    EVAL_STEPS = 50
    SAVE_STEPS = 50
    SAVE_LIMIT = 3
    WORKERS = 4
config = Config()
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

In [29]:
# -------------------------------------------------
# CELL 5: Load dataset
# -------------------------------------------------
print("Loading dataset...")
dataset = load_from_disk(config.HF_DATASET_PATH)

if isinstance(dataset, DatasetDict):
    train_ds = dataset["train"]
    val_ds   = dataset.get("validation", dataset.get("test"))
else:
    split = dataset.train_test_split(test_size=0.2, seed=42)
    train_ds, val_ds = split["train"], split["test"]

if not config.USE_FULL_DATASET:
    train_ds = train_ds.select(range(config.SMOKE_TRAIN))
    val_ds   = val_ds.select(range(config.SMOKE_VAL))

print(f"Raw → {len(train_ds)} train, {len(val_ds)} val\n")

Loading dataset...
Raw → 4445 train, 1133 val



In [30]:
# -------------------------------------------------
# CELL 6: Fix audio paths
# -------------------------------------------------
def fix_path(ex):
    fn = os.path.basename(ex["audio_filepath"])
    p  = os.path.join(config.AUDIO_SEGMENTS_PATH, fn)
    return {"audio_filepath": p if os.path.exists(p) else None}

train_ds = train_ds.map(fix_path, desc="Fix train", keep_in_memory=True, load_from_cache_file=False)
val_ds   = val_ds.map(fix_path,   desc="Fix val",   keep_in_memory=True, load_from_cache_file=False)

train_ds = train_ds.filter(lambda x: x["audio_filepath"] is not None)
val_ds   = val_ds.filter(lambda x: x["audio_filepath"] is not None)

print(f"After path fix → {len(train_ds)} train, {len(val_ds)} val\n")

Fix train:   0%|          | 0/4445 [00:00<?, ? examples/s]

Fix val:   0%|          | 0/1133 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4445 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1133 [00:00<?, ? examples/s]

After path fix → 4445 train, 1133 val



In [31]:
# -------------------------------------------------
# CELL 7: Processor + expected length
# -------------------------------------------------
processor = WhisperProcessor.from_pretrained(config.MODEL_NAME)

tmp = WhisperForConditionalGeneration.from_pretrained(config.MODEL_NAME)
expected_seq_len = int(tmp.config.max_source_positions *
                       tmp.model.encoder.conv1.stride[0] *
                       tmp.model.encoder.conv2.stride[0])
n_mels = tmp.model.encoder.conv1.weight.shape[1]
del tmp
torch.cuda.empty_cache()
print(f"Expected seq len: {expected_seq_len}, mels: {n_mels}\n")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Expected seq len: 3000, mels: 80



In [32]:
# -------------------------------------------------
# CELL 8: Audio → features helper
# -------------------------------------------------
def read_audio(p):
    if not p:
        return np.zeros(16000, dtype=np.float32)
    try:
        audio, sr = sf.read(p, dtype="float32")
        if audio.ndim > 1: audio = np.mean(audio, axis=1)
        if sr != 16000:   audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        return audio
    except:
        return np.zeros(16000, dtype=np.float32)

def prepare(ex):
    audio = read_audio(ex["audio_filepath"])
    feat  = processor.feature_extractor(audio, sampling_rate=16000,
                                       return_tensors="np")["input_features"][0]

    
    if feat.shape[0] != n_mels:
        feat = feat.T

    # pad / truncate
    if feat.shape[1] > expected_seq_len:
        feat = feat[:, :expected_seq_len]
    else:
        pad = expected_seq_len - feat.shape[1]
        feat = np.pad(feat, ((0,0), (0,pad)), constant_values=0)

    labels = processor.tokenizer(ex["text"], return_tensors="np",
                                 add_special_tokens=True).input_ids[0]

    return {
        "input_features": feat.astype(np.float32),
        "labels":         labels.astype(np.int64)
    }

print("Prep functions ready\n")

Prep functions ready



In [34]:
# -------------------------------------------------
# CELL 9: Process (single-process, Kaggle-safe)
# -------------------------------------------------
print("Processing datasets (single-process)…\n")
train_proc = train_ds.map(prepare, remove_columns=train_ds.column_names,
                          desc="Train", keep_in_memory=True,
                          load_from_cache_file=False, num_proc=1)
val_proc   = val_ds.map(prepare,   remove_columns=val_ds.column_names,
                        desc="Val",   keep_in_memory=True,
                        load_from_cache_file=False, num_proc=1)

print(f"Processed → {len(train_proc)} train, {len(val_proc)} val\n")

Processing datasets (single-process)…



Train:   0%|          | 0/4445 [00:00<?, ? examples/s]

Val:   0%|          | 0/1133 [00:00<?, ? examples/s]

Processed → 4445 train, 1133 val



In [35]:
# -------------------------------------------------
# CELL 10: Data collator
# -------------------------------------------------
@dataclass
class Collator:
    processor: Any
    def __call__(self, batch):
        inp = [{"input_features": b["input_features"]} for b in batch]
        batch_feat = self.processor.feature_extractor.pad(inp, return_tensors="pt")

        lbl = [{"input_ids": b["labels"]} for b in batch]
        batch_lbl  = self.processor.tokenizer.pad(lbl, return_tensors="pt")
        labels = batch_lbl["input_ids"].masked_fill(batch_lbl.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all():
            labels = labels[:, 1:]

        batch_feat["labels"] = labels
        return batch_feat

collator = Collator(processor=processor)
print("Collator ready\n")

Collator ready



In [36]:
# -------------------------------------------------
# CELL 11: MODEL + LoRA + PEFT FIX (NO OOM, NO ERROR)
# -------------------------------------------------
print("Loading Whisper-small + LoRA (P100 FINAL VERSION)...")

model = WhisperForConditionalGeneration.from_pretrained(
    config.MODEL_NAME,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)
model.to("cuda")


model.config.use_cache = False
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


model.gradient_checkpointing_enable()


from peft import PeftModel, LoraConfig, get_peft_model, prepare_model_for_kbit_training

class WhisperPeftFix(PeftModel):
    def forward(self, input_features=None, labels=None, **kwargs):
        # Remove args Whisper doesn't accept
        for key in ["input_ids", "attention_mask", "token_type_ids"]:
            kwargs.pop(key, None)
        return self.base_model(input_features=input_features, labels=labels, **kwargs)

# Apply LoRA
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=config.LORA_R,
    lora_alpha=config.LORA_ALPHA,
    target_modules=config.TARGET_MOD,
    lora_dropout=config.LORA_DROP,   # 0.0 = no OOM
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)
model = get_peft_model(model, lora_config)
model.__class__ = WhisperPeftFix   

print("Model ready – gradient checkpointing ON, LoRA dropout=0")
model.print_trainable_parameters()
print(f"Estimated VRAM: ~12.8 GB (safe on P100)")

Loading Whisper-small + LoRA (P100 FINAL VERSION)...
Model ready – gradient checkpointing ON, LoRA dropout=0
trainable params: 5,308,416 || all params: 247,043,328 || trainable%: 2.148779342868956
Estimated VRAM: ~12.8 GB (safe on P100)


In [37]:
# -------------------------------------------------
# CELL 12: Training Args – FINAL SAFE VERSION
# -------------------------------------------------
training_args = Seq2SeqTrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.NUM_EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=config.GRAD_ACC,
    learning_rate=config.LR,
    warmup_ratio=config.WARMUP,
    fp16=True,
    evaluation_strategy="steps",
    eval_steps=config.EVAL_STEPS,
    save_strategy="steps",
    save_steps=config.SAVE_STEPS,
    save_total_limit=config.SAVE_LIMIT,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=config.LOG_STEPS,
    report_to=[],
    remove_unused_columns=False,
    label_names=["labels"],
    dataloader_num_workers=config.WORKERS,
    dataloader_pin_memory=True,
    seed=42,
    # Memory optimization
    gradient_checkpointing=True,        
    torch_compile=False,                
)

In [38]:
# -------------------------------------------------
# CELL 13: Trainer (standard – PEFT 0.6.0 works)
# -------------------------------------------------
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_proc,
    eval_dataset=val_proc,
    data_collator=collator,
    tokenizer=processor.feature_extractor,
)
print("Trainer ready\n")

Trainer ready



In [41]:
# -------------------------------------------------
# CELL 14: TRAIN (now works 100%) – FIXED SYNTAX
# -------------------------------------------------
print("="*70)
print("STARTING TRAINING – P100 OPTIMIZED")
print("="*70)
print(f"Samples: {len(train_proc)} train | {len(val_proc)} val")
print(f"Batch: {config.BATCH_SIZE} | Epochs: {config.NUM_EPOCHS}")
print(f"Steps ≈ {(len(train_proc)//config.BATCH_SIZE)*config.NUM_EPOCHS}")
print("="*70)

ckpts = [d for d in os.listdir(config.OUTPUT_DIR) if d.startswith("checkpoint-")]
resume_path = None
if ckpts:
    
    latest = sorted(ckpts, key=lambda x: int(x.split("-")[1]))[-1]
    resume_path = os.path.join(config.OUTPUT_DIR, latest)
    print(f"Resuming from {latest}")

try:
    trainer.train(resume_from_checkpoint=resume_path)
    print("\nTRAINING COMPLETE!")
except KeyboardInterrupt:
    print("\nTraining stopped by user – checkpoint already saved.")
except Exception as e:
    print(f"\nTraining failed: {e}")
    raise

STARTING TRAINING – P100 OPTIMIZED
Samples: 4445 train | 1133 val
Batch: 24 | Epochs: 3
Steps ≈ 555


Step,Training Loss,Validation Loss
50,0.879700,0.826382
100,0.557700,0.506465
150,0.482300,0.441996
200,0.411000,0.418525
250,0.385200,0.402421
300,0.375600,0.385854
350,0.336100,0.369083
400,0.276400,0.360934
450,0.286600,0.355845
500,0.253800,0.349365



TRAINING COMPLETE!


In [42]:
# -------------------------------------------------
# CELL 15: Save final model
# -------------------------------------------------
final_dir = os.path.join(config.OUTPUT_DIR, "final_model")
os.makedirs(final_dir, exist_ok=True)
model.save_pretrained(final_dir)
processor.save_pretrained(final_dir)

if 'result' in locals():
    with open(os.path.join(config.OUTPUT_DIR, "metrics.json"), "w") as f:
        json.dump(result.metrics, f, indent=2)

print(f"Final model saved to {final_dir}\n")

Final model saved to /kaggle/working/whisper_lora_output/final_model



In [43]:
#-------------------------------------------------
# CELL 16: Final evaluation
# -------------------------------------------------
eval_res = trainer.evaluate()
print("\nEvaluation results:")
for k, v in eval_res.items():
    print(f"  {k}: {v:.4f}")
print()


Evaluation results:
  eval_loss: 0.3473
  eval_runtime: 108.0308
  eval_samples_per_second: 10.4880
  eval_steps_per_second: 0.4440
  epoch: 3.0000



In [46]:
# -------------------------------------------------
# CELL 17: Inference test – FINAL VERSION (NO MORE MIXED PRECISION ERRORS)
# -------------------------------------------------
print("Inference test (3 samples)…\n")


model.eval()


model = model.half()          # ← All weights/biases → FP16
model.to("cuda")

with torch.no_grad():
    for i in range(min(3, len(val_proc))):
        # Get input features and convert to FP16 immediately
        feat = torch.tensor(val_proc[i]["input_features"], dtype=torch.float16)
        feat = feat.unsqueeze(0).to("cuda")   # shape: [1, 80, 3000]

        generated_ids = model.generate(
            feat,
            max_length=128,
            num_beams=1,
            language="hi",
            task="transcribe",
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="hi", task="transcribe")
        )

        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(f"{i+1}. {transcription}\n")

print("="*70)
print("ALL DONE! Your Hindi Whisper LoRA model is working perfectly!")
print("="*70)
print(f"Download folder: {config.OUTPUT_DIR}/final_model\n")

Inference test (3 samples)…

1. तो मैंने अ पर्जुबार में उसके बारे में बताया कि मैं ये नौकरी अ छोड़ रहा हूं क्योंकि उसरे में काफी समय से एक व्र्क कर रहा था

2. तो मैंने परिवार को उसके बारे में जानकारी दी लेकिन अ परिवार में सहमत नहीं रहे थे कह रहे थे कि अभी और करो लेकिन मुझे अगले जॉब मे

3. और शार्शात में अपने जो हमारे बॉस होते थे उनको भी इसके बारे में जानकारी दी

ALL DONE! Your Hindi Whisper LoRA model is working perfectly!
Download folder: /kaggle/working/whisper_lora_output/final_model



In [47]:
# CELL: Merge LoRA into full model (run once!)
from peft import PeftModel


model = WhisperForConditionalGeneration.from_pretrained(
    "openai/whisper-small",
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, "/kaggle/working/whisper_lora_output/final_model")


merged_model_dir = "/kaggle/working/whisper-small-hindi-merged"
model = model.merge_and_unload()  
model.save_pretrained(merged_model_dir)
processor.save_pretrained(merged_model_dir)

print(f"Full merged model saved to: {merged_model_dir}")
print("Now download this folder — it's a complete standalone model!")

Full merged model saved to: /kaggle/working/whisper-small-hindi-merged
Now download this folder — it's a complete standalone model!


In [48]:

import shutil
from IPython.display import FileLink


shutil.make_archive(
    base_name="/kaggle/working/whisper-small-hindi-merged", 
    format='zip', 
    root_dir="/kaggle/working/whisper-small-hindi-merged"
)


FileLink("/kaggle/working/whisper-small-hindi-merged.zip")

/kaggle/working/whisper-small-hindi-merged.zip